# Библиотеки

In [1]:
import shutil
from openpyxl import *
from tkinter import *
from tkinter import ttk
from tkinter.ttk import Combobox
from tkinter import filedialog as fd
from tkcalendar import DateEntry, Calendar
from ttkwidgets.autocomplete import AutocompleteCombobox

# Функции

In [2]:
def check_input(event):
    value = event.widget.get()

    if len(event.keysym) == 1:
        event.widget.autocomplete()

    if value == '':
        Post_cb['value'] = Post_List
    else:
        data = []
        for item in Post_List:
            if value.lower() in item[0:len(value)].lower():
                data.append(item)
        Post_cb['value'] = data

# Основное окно

In [3]:
class main:
            
    def __init__(self, master):
            
        self.master = master
        self.master.title("Унифицированная форма № ОП-13") #Общее окно
        self.master.geometry("900x500") #Размер окна
        
        self.Name = Frame(self.master) #Наименование документа
        self.Name.place(x = 300, y = 10) #Положение рамки
        self.Doc_Name_lb = Label(self.Name, text = "Контрольный расчёт расхода специй и соли") #Название рамки 
        self.Doc_Name_lb.grid(row=1, column=1)
        
        self.Date = Frame(self.master) #Данные  о документе
        self.Date.place(x = 10, y = 40)
        self.Number_lb = Label(self.Date, text = "Номер документа")
        self.Number_lb.grid(row = 1, column = 1)
        self.Number_tf = Entry(self.Date, width=15)
        self.Number_tf.grid(row = 1, column = 2)
        self.Date_lb = Label(self.Date, text = "Дата составления")
        self.Date_lb.grid(row = 1, column = 3)
        self.Date_tf = DateEntry(self.Date, date_pattern = 'dd-mm-Y')
        self.Date_tf.delete(0, "end")
        self.Date_tf.grid(row = 1, column = 4)
        
        self.Organization = Frame(self.master) #Данные об организации
        self.Organization.place(x = 510, y = 40) #Положение рамки
        self.Organization_lb = Label(self.Organization, text = "Организация") #Название рамки
        self.Organization_lb.grid(row=1, column=1, sticky = "w")
        self.Organization_tf = Entry(self.Organization, width=35) #Ввод названия организации 
        self.Organization_tf.grid(row = 1, column = 2)
        self.Subdivision_lb = Label(self.Organization, text = "Структурное подразделение")
        self.Subdivision_lb.grid(row=2, column=1)
        self.Subdivision_tf = Entry(self.Organization, width=35) #Ввод названия подразделения 
        self.Subdivision_tf.grid(row = 2, column = 2)
        
        self.Tables = Frame(self.master)
        self.Tables.place(x = 10, y = 123)
        columns = ("number", "name", "start_balance", "received", "end_balance", "used")

        self.tree = ttk.Treeview(self.Tables, columns=columns, show="headings", height = 8)
        self.tree.pack(fill=BOTH, expand=1)
        
        style = ttk.Style()
        style.configure('Treeview.Heading', foreground='black', background='white', font=('Arial',7),)
        self.tree.heading("#1", text="№", anchor="center", command=lambda: self.sort(0, False))
        self.tree.heading("#2", text="Наименование", anchor="center", command=lambda: self.sort(1, False))
        self.tree.heading("#3", text="Остаток на начало отчетного \n периода (сумма), руб. коп.",
                          anchor="center", command=lambda: self.sort(2, False))
        self.tree.heading("#4", text="Поступило за отчетный \n период (сумма), руб. коп.",
                          anchor="center", command=lambda: self.sort(3, False))
        self.tree.heading("#5", text="Остаток на конец отчетного \n периода (сумма), руб. коп.",
                          anchor="center", command=lambda: self.sort(4, False))
        self.tree.heading("#6", text="Израсходовано за отчетный \n период (сумма), руб. коп.",
                          anchor="center", command=lambda: self.sort(5, False))
        
        self.tree.column("#1", width=31)
        self.tree.column("#2", width=102)
        self.tree.column("#3", width=186)
        self.tree.column("#4", width=186)
        self.tree.column("#5", width=186)
        self.tree.column("#6", width=186)
        
        self.tree.insert("", END, values = (1, "Соль"))
        self.tree.insert("", END, values = (2, "Перец"))
        self.tree.insert("", END, values = (3, "Горчица"))
        
        Add_logo = PhotoImage(file="./Add.png")
        Delet_logo = PhotoImage(file="./Delet.png")
        Edite_logo = PhotoImage(file="./Edite.png")
        Save_logo = PhotoImage(file="./Save.png")
        
        self.Buttons = Frame(self.master)
        self.Buttons.place(x = 801, y = 310)
        self.Add_bt = Button(self.Buttons, image = Add_logo, command = self.add)
        self.Add_bt.grid(row = 1, column = 1)
        self.Add_bt = Button(self.Buttons, image = Delet_logo, command = self.dellet)
        self.Add_bt.grid(row = 1, column = 2)
        self.Add_bt = Button(self.Buttons, image = Edite_logo, command = self.edit)
        self.Add_bt.grid(row = 1, column = 3)
        self.Add_bt = Button(self.Buttons, image = Save_logo, command = self.out)
        self.Add_bt.grid(row = 1, column = 4)
        
        self.Reference = Frame(self.master)
        self.Reference.place(x = 10, y = 330)
        self.Reference_tx = Label(self.Reference, text="Справка о стоимости специй и соли, включенной в калькуляцию блюд:")
        self.Reference_tx.grid(row = 1, column = 1)
        
        self.Eight = Frame(self.master)
        self.Eight.place(x = 10, y = 360)
        self.Eight_lb = Label(self.Eight, text = "Стоимость специй на блюдо")
        self.Eight_lb.grid(row = 1, column = 1)
        self.Eight_tf = Entry(self.Eight, width=13)
        self.Eight_tf.grid(row = 1, column = 2)
        self.Eight_tf.insert(0, 0.0)
        self.Eight_lb2 = Label(self.Eight, text = " руб. коп. ")
        self.Eight_lb2.grid(row = 1, column = 3)
        self.Eight_lb4 = Label(self.Eight, text = "Стоимость соли на блюдо")
        self.Eight_lb4.grid(row = 2, column = 1, sticky = "w")
        self.Eight_tf4 = Entry(self.Eight, width=13)
        self.Eight_tf4.grid(row = 2, column = 2)
        self.Eight_tf4.insert(0, 0.0)
        self.Eight_lb5 = Label(self.Eight, text = " руб. коп.")
        self.Eight_lb5.grid(row = 2, column = 3)
        
        self.DishCalc = Frame(self.master)
        self.DishCalc.place(x = 320, y = 360)
        self.DishCalc_lb = Label(self.DishCalc, text = "Количество блюд", width=15)
        self.DishCalc_lb.grid(row = 1, column = 1)
        self.DishCalc_tf = Entry(self.DishCalc, width=13)
        self.DishCalc_tf.grid(row = 1, column = 2)
        self.DishCalc_tf.insert(0, 0)
        
        self.DishCalc_bt2 = Button(self.DishCalc, text = "Сумма, руб. коп.", command = self.sum, width=13)
        self.DishCalc_bt2.grid(row = 2, column = 1)
        self.DishCalc_tf2 = Entry(self.DishCalc, width=13)
        self.DishCalc_tf2.grid(row = 2, column = 2)
        
        self.Total = Frame(self.master)
        self.Total.place(x = 10, y = 410)
        self.Total_tx = Label(self.Total, text="Итого:")
        self.Total_tx.grid(row = 1, column = 1)
        
        self.Eight2 = Frame(self.master)
        self.Eight2.place(x = 10, y = 430)
        self.Eight2_lb = Label(self.Eight2, text = "Израсходовано согласно контрольного расчета")
        self.Eight2_lb.grid(row = 1, column = 1)
        self.Eight2_tf = Entry(self.Eight2, width=13)
        self.Eight2_tf.grid(row = 1, column = 3)
        self.Eight2_lb2 = Label(self.Eight2, text = "Сумма недорасхода")
        self.Eight2_lb2.grid(row = 2, column = 1, sticky = "w")
        self.Eight2_tf2 = Entry(self.Eight2, width=13)
        self.Eight2_tf2.grid(row = 2, column = 3)
        
        self.master.mainloop()
        
    def sort(self, col, reverse):
        l = [(self.tree.set(k, col), k) for k in self.tree.get_children("")]
        l.sort(reverse=reverse)
        
        for index,  (_, k) in enumerate(l):
            self.tree.move(k, "", index)
            
        self.tree.heading(col, command=lambda: self.sort(col, not reverse))
        
    def sum(self):
        self.DishCalc_tf2.delete(0, END)
        self.DishCalc_tf2.insert(0, round((float(self.Eight_tf.get()) + float(self.Eight_tf4.get()))
                                          * int(self.DishCalc_tf.get()), 2))
        
        child = self.tree.get_children()
        N1 = self.tree.item(child[0])["values"]
        N2 = self.tree.item(child[1])["values"]
        N3 = self.tree.item(child[2])["values"]
        
        self.Eight2_tf.delete(0, END)
        self.Eight2_tf.insert(0, float(N1[5]) + float(N2[5]) + float(N3[5]))
        
        self.Eight2_tf2.delete(0, END)
        self.Eight2_tf2.insert(0, round((float(N1[4]) + float(N2[4]) + float(N3[4])) - float(self.Eight2_tf.get()), 2))
    
    def add(self):
        self.dialog = child(self.master)
        self.returnValue = self.dialog.go()
        if self.returnValue:
            self.tree.insert("", END, values = self.returnValue)
            
    def dellet(self):
        item = self.tree.selection()[0]
        self.tree.delete(item)
        
    def edit(self):
        select = self.tree.focus()
        temp = self.tree.item(select, 'values')
        while(len(temp) < 5):
            temp = list(temp)
            temp.insert(len(temp), 0)
            temp = tuple(temp)
        self.dialog = child(self.master)
        self.sendValue = temp
        self.returnValue = self.dialog.go_edit(temp)
        if self.returnValue:
            self.tree.delete(select)
            self.tree.insert("", END, values = self.returnValue)
            
    def out(self):
        filetypes = (('Книга Excel', '*.xlsx'),('Книга Excel 97—2003', '*.xls'),('Все файлы', '*.*'))
        shutil.copyfile('C:\\Users\\Alfatsentavra\\Lab ASTU\\1 course\\2 semester\\DUI\\Lab 1\\Shablon.xlsx',
                        'C:\\Users\\Alfatsentavra\\Lab ASTU\\1 course\\2 semester\\DUI\\Lab 1\\Form13.xlsx')
        Form = load_workbook('C:\\Users\\Alfatsentavra\\Lab ASTU\\1 course\\2 semester\\DUI\\Form13.xlsx')
        sheet = Form.active
        cell = sheet['A6']
        cell.value = self.Organization_tf.get()
        cell = sheet['A8']
        cell.value = self.Subdivision_tf.get()
        cell = sheet['Q16']
        cell.value = self.Number_tf.get()
        cell = sheet['W16']
        cell.value = self.Date_tf.get()
        
        child = self.tree.get_children()
        N1 = self.tree.item(child[0])["values"]
        N2 = self.tree.item(child[1])["values"]
        N3 = self.tree.item(child[2])["values"]
        
        cell = sheet['W25']
        cell.value = N1[2]
        cell = sheet['AD25']
        cell.value = N1[3]
        cell = sheet['AK25']
        cell.value = N1[4]
        cell = sheet['AS25']
        cell.value = N1[5]
        
        cell = sheet['W26']
        cell.value = N2[2]
        cell = sheet['AD26']
        cell.value = N2[3]
        cell = sheet['AK26']
        cell.value = N2[4]
        cell = sheet['AS26']
        cell.value = N2[5]
        
        cell = sheet['W27']
        cell.value = N3[2]
        cell = sheet['AD27']
        cell.value = N3[3]
        cell = sheet['AK27']
        cell.value = N3[4]
        cell = sheet['AS27']
        cell.value = N3[5]
        
        cell = sheet['W32']
        cell.value = float(N1[2]) + float(N2[2]) + float(N3[2])
        cell = sheet['AD32']
        cell.value = float(N1[3]) + float(N2[3]) + float(N3[3])
        cell = sheet['AK32']
        cell.value = float(N1[4]) + float(N2[4]) + float(N3[4])
        cell = sheet['AS32']
        cell.value = float(N1[5]) + float(N2[5]) + float(N3[5])
        
        cell = sheet['C40']
        Pr = self.Eight_tf.get()
        Price = Pr.split(".")
        cell.value = Price[0]
        cell = sheet['T40']
        cell.value = Price[1]
        
        cell = sheet['C43']
        Pr = self.Eight_tf4.get()
        Price = Pr.split(".")
        cell.value = Price[0]
        cell = sheet['T43']
        cell.value = Price[1]
        
        cell = sheet['AE39']
        cell.value = self.DishCalc_tf.get()
        cell = sheet['AE42']
        cell.value = self.DishCalc_tf.get()
        cell = sheet['AE45']
        cell.value = self.DishCalc_tf.get()
        
        cell = sheet['AL39']
        cell.value = float(self.Eight_tf.get()) * int(self.DishCalc_tf.get())
        cell = sheet['AL42']
        cell.value = float(self.Eight_tf4.get()) * int(self.DishCalc_tf.get())
        cell = sheet['AL45']
        cell.value = self.DishCalc_tf2.get()
        
        cell = sheet['AE46']
        cell.value = self.Eight2_tf.get()
        cell = sheet['AE47']
        cell.value = self.Eight2_tf2.get()
        
        path = fd.asksaveasfilename(filetypes = filetypes, defaultextension = "xlsx")
        
        if path != "":
            Form.save(path)

# Дочернее окно

In [4]:
class child:
    def __init__(self, master):
        self.slave = Toplevel(master)
        self.slave.title("Добавление элемента")
        self.slave.geometry("260x260")
        
        self.Data = Frame(self.slave)
        self.Data.place(x = 10, y = 10)
        self.Numder_add = Label(self.Data, text = "№")
        self.Numder_add.grid(row = 1, column = 1, sticky = "w")
        self.Numder_add_tf = Entry(self.Data)
        self.Numder_add_tf.grid(row = 1, column = 2)

        self.Name_add = Label(self.Data, text = "Наименование")
        self.Name_add.grid(row = 2, column = 1, sticky = "w")
        self.Name_add_tf = Entry(self.Data)
        self.Name_add_tf.grid(row = 2, column = 2)

        self.Statr_balance = Label(self.Data, text = "Остаток на начало \n отчетного периода \n (сумма), руб. коп.")
        self.Statr_balance.grid(row = 3, column = 1, sticky = "w")
        self.Statr_balance_tf = Entry(self.Data)
        self.Statr_balance_tf.grid(row = 3, column = 2)

        self.Balance_add = Label(self.Data, text = "Поступило за \n отчетный период \n (сумма), руб. коп.")
        self.Balance_add.grid(row = 4, column = 1, sticky = "w")
        self.Balance_add_tf = Entry(self.Data)
        self.Balance_add_tf.grid(row = 4, column = 2)

        self.End_balance = Label(self.Data, text = "Остаток на конец \n отчетного периода \n (сумма), руб. коп.")
        self.End_balance.grid(row = 5, column = 1, sticky = "w")
        self.End_balance_tf = Entry(self.Data)
        self.End_balance_tf.grid(row = 5, column = 2)

        self.Buttons_add = Frame(self.slave)
        self.Buttons_add.place(x = 10, y = 220)
        self.Ok_butoon = Button(self.Buttons_add, text = "ОК", command = self.output)
        self.Ok_butoon.grid(row = 1, column = 1)

        self.Empty = Label(self.Buttons_add, width = 22)
        self.Empty.grid(row = 1, column = 2)
        self.Canlec_button = Button(self.Buttons_add, text = "Отмена", command=lambda: self.slave.destroy())
        self.Canlec_button.grid(row = 1, column = 3)
    
    def go(self):
        self.newValue = None
        
        self.slave.grab_set()
        self.slave.focus_set()
        self.slave.wait_window()
        
        return self.newValue
    
    def go_edit(self, temp = ()):
        self.newValue = None
        
        self.Numder_add_tf.insert(0, temp[0])
        self.Name_add_tf.insert(0, temp[1])
        self.Statr_balance_tf.insert(0, temp[2])
        self.Balance_add_tf.insert(0, temp[3])
        self.End_balance_tf.insert(0, temp[4])
        
        self.slave.grab_set()
        self.slave.focus_set()
        self.slave.wait_window()
        
        return self.newValue
    
    def output(self):
        self.Number = self.Numder_add_tf.get()
        self.Name = self.Name_add_tf.get()
        self.S_Balance = self.Statr_balance_tf.get()
        self.A_Balance = self.Balance_add_tf.get()
        self.E_Balance = self.End_balance_tf.get()
        self.Consumption = round(float(self.S_Balance) + float(self.A_Balance) - float(self.E_Balance), 2)

        self.Record = (self.Number, self.Name, self.S_Balance, self.A_Balance, self.E_Balance, self.Consumption)
        self.newValue = self.Record
    
        self.slave.destroy()

# Вызов

In [5]:
root = Tk()

In [6]:
main(root)